In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Amandeep
[nltk_data]     Sandhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('ecommerce_sample_dataset.csv')
df_new=df[['product_url','product_name','product_category_tree','discounted_price','is_FK_Advantage_product','description','overall_rating','brand','product_specifications']]
bins = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,20000,30000,40000,50000]
labels = ["<1000", "1000-2000", "2000-3000", "3000-4000", "4000-5000", "5000-6000", "6000-7000", "7000-8000", "8000-9000", "9000-1000","10000-20000", "20000-30000", "30000-40000", ">50000"]
df_new['price_bracket'] = pd.cut(df_new['discounted_price'], bins, labels=labels)
df_new=df_new.dropna()

def combine_features(row):
        try:
            return row['product_name']+" "+str(row['discounted_price'])+" "+str(row['is_FK_Advantage_product'])+" "+str(row['overall_rating'])+" "+row['brand']+" "+row['product_specifications']+" "+str(row['price_bracket'])
        except:
            print("Error",row)
            
    
df_new["combined_features"] = df_new.apply(combine_features,axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [3]:
df_new.to_csv('cleaned_data.csv') 

In [4]:
i=df_new.shape[0]
l=list(range(0,i)) 
df_new.index=l

In [5]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_new['combined_features'])
tfidf_matrix.shape

(14040, 292235)

In [6]:
df1=df_new['product_url']
df1.size

14040

In [7]:
def item(id):
    return df1[id]

def recommend(indices,scores,item_id,num):
    print("Recommending " + str(num) + " products similar to " + item(item_id))
    print("---------------")
    i=0
    while(i<num):
        indice=indices[0,i+1]
        print("Recommended: " + item(indice)+" Score: "+  str(scores[0,indice]))
        i=i+1

def similar(item_id, num):
    results = []
    cosine_similarities = linear_kernel(tfidf_matrix[item_id], tfidf_matrix)
    similar_indices = (-cosine_similarities).argsort()
    recommend(similar_indices,cosine_similarities,item_id,num)

In [46]:
val = input("Enter index of item(check file [cleaned_data.csv] for index): ") 

Enter index of item(check file [cleaned_data.csv] for index): 9979


In [47]:
similar(int(val),5)

Recommending 5 products similar to http://www.flipkart.com/karatcraft-risena-gold-diamond-18-k-ring/p/itme87fchrqpkgpc?pid=RNGE87FCSCBAVHFT
---------------
Recommended: http://www.flipkart.com/karatcraft-trilic-gold-diamond-18-k-ring/p/itme87fcfefhrq5a?pid=RNGE87FCSVHHR2SQ Score: 0.648468477154659
Recommended: http://www.flipkart.com/karatcraft-risena-gold-diamond-18-k-ring/p/itme7t8ffacgy3hc?pid=RNGE86J7VJNYBSHZ Score: 0.6472100757674706
Recommended: http://www.flipkart.com/karatcraft-twilite-gold-diamond-18-k-ring/p/itme87fca6xgahgh?pid=RNGE87FCCDJQYSQ9 Score: 0.6420418242990369
Recommended: http://www.flipkart.com/karatcraft-klytie-gold-diamond-18-k-ring/p/itme87fca9ahazbs?pid=RNGE87FCKGMNGHXP Score: 0.6095711172647118
Recommended: http://www.flipkart.com/karatcraft-raciel-gold-diamond-18-k-ring/p/itme87fcptmgy2yh?pid=RNGE87FCHNTBK927 Score: 0.6081185383037092
